# Clear sentences from ingredients

In [348]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim
import numpy as np
import gensim.downloader
import re
import pandas as pd
from pandas import DataFrame as df

In [8]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')
vector = wv['computer']  # Get numpy vector of a word
vector

array([-0.00515774, -0.00667028, -0.0077791 ,  0.00831315, -0.00198292,
       -0.00685696, -0.0041556 ,  0.00514562, -0.00286997, -0.00375075,
        0.0016219 , -0.0027771 , -0.00158482,  0.0010748 , -0.00297881,
        0.00852176,  0.00391207, -0.00996176,  0.00626142, -0.00675622,
        0.00076966,  0.00440552, -0.00510486, -0.00211128,  0.00809783,
       -0.00424503, -0.00763848,  0.00926061, -0.00215612, -0.00472081,
        0.00857329,  0.00428458,  0.0043261 ,  0.00928722, -0.00845554,
        0.00525685,  0.00203994,  0.0041895 ,  0.00169839,  0.00446543,
        0.00448759,  0.0061063 , -0.00320303, -0.00457706, -0.00042664,
        0.00253447, -0.00326412,  0.00605948,  0.00415534,  0.00776685,
        0.00257002,  0.00811904, -0.00138761,  0.00808028,  0.0037181 ,
       -0.00804967, -0.00393476, -0.0024726 ,  0.00489447, -0.00087241,
       -0.00283173,  0.00783599,  0.00932561, -0.0016154 , -0.00516075,
       -0.00470313, -0.00484746, -0.00960562,  0.00137242, -0.00

In [359]:
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[==================================================] 100.0% 66.0/66.0MB downloaded


In [312]:
print("ok")

ok


In [344]:
print(glove_vectors.most_similar('eggs'))
glove_vectors.cosine_similarities(glove_vectors['prosciutto'], [glove_vectors['ham'], glove_vectors['salt']])

[('egg', 0.8321493864059448), ('cooked', 0.591994047164917), ('nest', 0.5881955623626709), ('fertilized', 0.5780420303344727), ('eaten', 0.5704600214958191), ('yolk', 0.5473426580429077), ('eat', 0.5452857613563538), ('chicks', 0.5437774658203125), ('chicken', 0.5409024953842163), ('nests', 0.5403977632522583)]


array([0.44192395, 0.18894038], dtype=float32)

In [314]:
def preprocess(line):
	res = line.lower()
	res = res.replace('(', '').replace(')', '').replace('/', ' ').replace('-', ' ').replace('\'', '').replace('\"', '').replace('?', '').replace('!', '').replace(',', '').replace(';', '').replace(':', '')
	res = res.replace(' one ', ' 1 ').replace(' two ', ' 2 ').replace(' three ', ' 3 ').replace(' four ', ' 4 ').replace(' five ', ' 5 ').replace(' six ', ' 6 ').replace(' seven ', ' 7 ').replace(' eight ', ' 8 ').replace(' nine ', ' 9 ').replace(' ten ', ' 10 ')
	res = " ".join(re.split(',| |\|/', res))

	mults = re.findall(r"[0-9]+[\s]*[x,\\,+][\s]*[0-9]+", res)
	for m in mults:
		res = res.replace(m, str(eval(m.replace('x', '*'))))
	return res


In [315]:
ingr_ex_1 = ['a good beef fillet (preferably Aberdeen Angus) of around 1kg/2lb 4oz','3 tbsp olive oil','250g/9oz chestnut mushroom, include some wild ones if you like','50g/2oz butter','1 large sprig fresh thyme','100ml/3.5 fl oz dry white wine','2 x 6 slices prosciutto','1+1kg/1lb 2oz pack puff pastry, thawed if frozen','a little flour, for dusting','2 egg yolks beaten with 1 tsp water']
ingr_ex_2 = ['1/2 cup (125 mL) unsalted butter, softened','1/2 cup (125 mL) granulated sugar','1/2 cup (125 mL) packed brown sugar','1 large egg','1 tsp (5 mL) vanilla extract','1 1/2 cups (375 mL) all-purpose flour','1/2 tsp (2 mL) baking soda','1/2 tsp (2 mL) salt','1 cup (250 mL) old-fashioned rolled oats','1 cup (250 mL) semisweet chocolate chips','1 cup (250 mL) white chocolate chips','1 cup (250 mL) dried cranberries','1 cup (250 mL) chopped pecans']
ingr_ex_3 = ['650ml milk', '40g plain flour', '40g butter', '2 tsp Dijon mustard', '150g mature  cheddar, grated', '180g frozen peas', 'handful of  parsley, chopped', '300g macaroni', '300g fish pie mix (smoked fish, white fish and salmon)', 'green salad, to serve (optional)']
ingr_ex_4 = ['1 tsp plain flour', '1 tsp mustard powder', '950g beef top rump joint (see tip below)', '1 onion, cut into 8 wedges', '500g carrots, halved lengthways', '1 tbsp plain flour', '250ml beef stock']


In [316]:
food_data = pd.read_csv('fooddata.csv')
food_data = food_data.dropna()
print(len(food_data))
print(food_data.head())

food_dict = {}
for name, serving, per100 in zip(food_data['Name'], food_data['Suggested_Serving'], food_data['per100g']):
	food_dict[name] = (serving, per100)
food_dict

524
   Unnamed: 0          Name  Suggested_Serving  per100g
0           0         bagel               85.0      121
1           1      baguette              220.0      130
2           2         bread               80.0      121
3           3  bread crumbs              112.0      121
4           4    bread roll               60.0      121


{'bagel': (85.0, 121),
 'baguette': (220.0, 130),
 'bread': (80.0, 121),
 'bread crumbs': (112.0, 121),
 'bread roll': (60.0, 121),
 'brown bread': (80.0, 110),
 'brown bread roll': (60.0, 121),
 'cake': (120.0, 306),
 'chocolate cake': (120.0, 367),
 'crepe': (80.0, 71),
 'crumpet': (50.0, 121),
 'doughnut': (75.0, 658),
 'fruit cake': (120.0, 306),
 'muffin': (130.0, 658),
 'pancake': (50.0, 71),
 'pastry': (47.0, 658),
 'pitta bread': (60.0, 121),
 'plain naan bread': (140.0, 121),
 'rye bread': (80.0, 107),
 'scones': (70.0, 306),
 'sponge cake': (120.0, 306),
 'tortilla wrap': (65.0, 108),
 'waffle': (75.0, 71),
 'white bread': (80.0, 108),
 'white bread roll': (60.0, 112),
 'fruit pie': (120.0, 81),
 'cheesecake': (120.0, 548),
 'biscuits': (45.0, 130),
 'chocolate': (30.0, 576),
 'chocolate biscuit': (45.0, 270),
 'chocolate chips': (10.0, 576),
 'chocolate snack bar': (32.0, 507),
 'crackers': (24.0, 116),
 'flapjacks': (32.0, 684),
 'fudge': (34.0, 241),
 'gluten-free biscuit'

In [327]:
def extract_amount(line):
	grams = re.findall(r"[0-9]+[\s]*[mk]*g ", line)
	litres = re.findall(r"[0-9]+[\s]*[m]*l ", line)
	litres = litres + re.findall(r"[0-9]+[\s]*[m]*litre", line)
	ounces = re.findall(r"[0-9]+[\s]*oz[s]* ", line)
	fl_ounces = re.findall(r"[0-9]+[\s]*fl oz[s]* ", line)
	pounds = re.findall(r"[0-9]+[\s]*lb[s]* ", line)
	teaspoons = re.findall(r"[0-9]+[\s]*[tT]sp[s]* ", line)
	tablespoons = re.findall(r"[0-9]+[\s]*tbsp[s]* ", line)
	cups = re.findall(r"[0-9]+[\s]*[cC]up[s]* ", line)
	pints = re.findall(r"[0-9]+[\s]*[pP]int[s]* ", line)
	quarts = re.findall(r"[0-9]+[\s]*[qQ]uart[s]* ", line)
	gallons = re.findall(r"[0-9]+[\s]*[gG]allon[s]* ", line)
	pieces = re.findall(r"[0-9]+[\s]*[pP]iece[s]* ", line)
	portions = re.findall(r"[0-9]+[\s]*[pP]ortion[s]* ", line)
	slice = re.findall(r"[0-9]+[\s]*[sS]lice[s]* ", line)

	numbers = re.findall(r"[0-9]+[\s]*", line)
	numbers = [item+"portion" for item in numbers]

	amounts = grams + litres + ounces + fl_ounces + pounds + teaspoons + tablespoons + cups + pints + quarts + gallons + pieces + portions
	amounts = numbers if amounts == [] else amounts
	if amounts == []: amounts = ['1 portion ']

	amounts = amounts[0].replace(' ', '')
	amounts = amounts[:-1] if amounts[-1]=='s' else amounts

	return amounts

def unify_amount(amount, product):
	amount = amount.replace('kg', '*1000')
	amount = amount.replace('mg', '*0.1')
	amount = amount.replace('ml', '')
	amount = amount.replace('litre', '*1000')
	amount = amount.replace('floz', '*28.3495')
	amount = amount.replace('oz', '*28.3495')
	amount = amount.replace('lb', '*453.592')
	amount = amount.replace('tsp', '*4.92892')
	amount = amount.replace('tbsp', '*14.7868')
	amount = amount.replace('cup', '*236.588')
	amount = amount.replace('pint', '*473.176')
	amount = amount.replace('quart', '*946.353')
	amount = amount.replace('gallon', '*3785.41')
	amount = amount.replace('piece', 'portion')
	amount = amount.replace('slice', 'portion')
	amount = amount.replace('g', '')
	amount = amount.replace('l', '*1000')

	# ! TODO: replace portion with sth smart
	portion_size = str(food_dict[product][0])
	amount = amount.replace('portion', '*'+portion_size)
	
	return eval(amount)

In [352]:
def extract_product(line):
	keywords = re.sub(r'\W+', ' ', line).split()
	applicable = []
	for key in food_dict.keys():
		if ' ' in key:
			if line.find(key) != -1:
				applicable.append(key)
		else:
			if key in keywords or key+'s' in keywords:
				applicable.append(key)
			if key[-1]=='s' and key[:-1] in keywords:
				applicable.append(key)

	max_sim  = 0
	best_ingredient = ''

	if applicable == []:
		for key in food_dict.keys():
			similarity = 0
			if key in glove_vectors:
				similarity = np.max(glove_vectors.cosine_similarities(glove_vectors[key], [glove_vectors[word] if word in glove_vectors else glove_vectors['troll'] for word in keywords]))
			if similarity > max_sim:
				max_sim = similarity
				best_ingredient = key
		return best_ingredient
	return max(applicable, key=len)

In [353]:
extract_amount('100ml/3.5 fl oz dry white wine')

'5floz'

In [358]:
ingredients = ingr_ex_1

for line in ingredients:
	print(line)

print('\n\n')

for i in range(len(ingredients)):
	ingr_ex_2[i] = preprocess(ingredients[i])

result = []

for i in range(len(ingredients)):
	product = extract_product(ingredients[i])
	amount = (unify_amount(extract_amount(ingredients[i]), product))
	
	result.append(round(amount*food_dict[product][1]/100))

print(result)

a good beef fillet (preferably Aberdeen Angus) of around 1kg/2lb 4oz
3 tbsp olive oil
250g/9oz chestnut mushroom, include some wild ones if you like
50g/2oz butter
1 large sprig fresh thyme
100ml/3.5 fl oz dry white wine
2 x 6 slices prosciutto
1+1kg/1lb 2oz pack puff pastry, thawed if frozen
a little flour, for dusting
2 egg yolks beaten with 1 tsp water



[39308, 112, 373, 553, 27, 194, 398, 373, 7, 0]
